In [1]:
# Fist install the library

#%pip install aepy

In [2]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np

from sklearn.metrics import accuracy_score
from keras import utils

from rapidae.data.datasets import load_MNIST
from rapidae.data.utils import evaluate, display_diff, add_noise
from rapidae.models.vq_vae.vq_vae_model import VQ_VAE
from rapidae.models.base.default_architectures import Encoder_Conv_VQ_MNIST, Decoder_Conv_VQ_MNIST
from rapidae.pipelines.training import TrainingPipeline

2023-12-23 01:07:49.589634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-12-23 01:07:52 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-23 01:07:52 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-23 01:07:52 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-23 01:07:52 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [4]:
train_data = dict(data=x_train.astype(float), labels=x_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [5]:
# Model creation
model = VQ_VAE(input_dim=(x_train.shape[1], x_train.shape[2]), 
            latent_dim=2, encoder=Encoder_Conv_VQ_MNIST, decoder=Decoder_Conv_VQ_MNIST, layers_conf=[32, 64])

#model.jit_compile = False

In [6]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=2)

trained_model = pipe(x=x_train.astype(float), y=y_train)

2023-12-23 01:07:53 [INFO]: +++ training_pipeline +++
2023-12-23 01:07:53 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-23_01-07-53
/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/keras/src/layers/layer.py:357: UserWarning: `build()` was called on layer 'encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/keras/src/backend/common/backend_utils.py:88: UserWarning: You might experience inconsistencies accross backends when calling conv transpose with kernel_size=3, stride=2, dilation_rate=1, padding=same, output_padding=1.
  warnings.warn(


Epoch 1/2


ValueError: Exception encountered when calling VQ_VAE.call().

[1mAttempt to convert a value (tensor(163.6844, grad_fn=<AddBackward0>)) with an unsupported type (<class 'torch.Tensor'>) to a Tensor.[0m

Arguments received by VQ_VAE.call():
  • x=torch.Tensor(shape=torch.Size([128, 28, 28, 1]), dtype=float32)

In [ ]:
import matplotlib.pyplot as plt

def show_subplot(original, reconstructed):
    plt.subplot(1, 2, 1)
    plt.imshow(original.squeeze() + 0.5)
    plt.title("Original")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(reconstructed.squeeze() + 0.5)
    plt.title("Reconstructed")
    plt.axis("off")

    plt.show()

In [ ]:
idx = np.random.choice(len(x_test), 9)

test_images = x_test[idx]
test_labels = y_test[idx]

print(test_images.shape)
print(x_test.shape)

#test_data = dict(data=test_images.astype(float), labels=test_labels)

reconstructions_test = trained_model.predict(test_images.astype(float))

print(reconstructions_test['recon'])

for test_image, reconstructed_image in zip(test_images, reconstructions_test['recon']):
    show_subplot(test_image, reconstructed_image)